# 必須課題
> コーパスを選択し、ベクトル空間モデルに基づいた検索を実現せよ.
> 3種類以上のクエリでの検索結果を示すこと.

In [1]:
import json
import numpy as np
import gensim
import pandas as pd
from scipy.spatial.distance import cosine

In [2]:
with open("../data/kyoto_results_100.json", "r") as f:
    docs = json.load(f)

In [3]:
# [START 検索ロジックの基本クラス]
class BasicEngine(object):
    def __init__(self, docs):
        self.docs = docs
        self.dictionary = gensim.corpora.Dictionary([doc["bow"].split() for doc in docs])
        self.DICT_SIZE = len(self.dictionary)
        
    def _calc_cossim(self, v, w):
        return 1.0 - cosine(v, w)

In [6]:
# [START TF-IDFによる検索ロジックのクラス実装]
class TfidfEngine(BasicEngine):
    def __init__(self, docs):
        super().__init__(docs)
        id_corpus = [self.dictionary.doc2bow(doc["bow"].split()) for doc in docs]
        self.tfidf_model = gensim.models.TfidfModel(id_corpus, normalize=False)
        tfidf_corpus = self.tfidf_model[id_corpus]
        self.tfidf_vectors = gensim.matutils.corpus2dense(tfidf_corpus, self.DICT_SIZE).T

    def _calc_scores(self, query):
        tfidf_q = self.tfidf_model[self.dictionary.doc2bow(query)]
        query_vec = gensim.matutils.corpus2dense([tfidf_q], self.DICT_SIZE).T[0]
        return [self._calc_cossim(query_vec, doc_vec) for doc_vec in self.tfidf_vectors]
    
    def search(self, query, n_results=5):
        assert type(query) == set, "クエリはset型で返す"
        scores = self._calc_scores(query)
        s = pd.Series(scores)
        return [
                (idx, score, self.docs[idx]["title"]) for idx, score in s.sort_values(ascending=False)[:n_results].iteritems()
                ]

In [7]:
# [START 3種類のクエリで検索結果を示す]
q1 = {"観光", "おすすめ"}
q2 = {"紅葉", "穴場"}
q3 = {"鴨川", "料理"}

tfidf_engine = TfidfEngine(docs)
res1 = tfidf_engine.search(q1)
res2 = tfidf_engine.search(q2)
res3 = tfidf_engine.search(q3)

[(60, 0.20026417328123602, '京都観光おすすめのグルメや人気の穴場'),
 (30, 0.13748758827691754, '京都観光 | 京都の観光を楽しむならきょうと情報版'),
 (35, 0.13569327203818871, '京都観光地おすすめランキング - コトログ京都'),
 (10, 0.13220926013527468, '旬の京都観光情報と 京都のおすすめ観光スポット | 京都観光総 ...'),
 (3, 0.13062828660736625, '京都観光のおすすめコースをガイド | 京都じっくり観光')]

## 行ったこと
1. 